In [1]:
#%%capture --no-stderr
#!pip install pyautogen~=0.1.0
#!pip install openai

In [12]:
import json, os

# Define the content to be written to the file
config_list = [
    #{
    #    "model": "gpt-3.5-turbo",
    #    "api_key": "_____________"
    #},
    {
        "api_key": os.environ.get("GROQ_API_KEY"),
        "model": "llama3-70b-8192",
        "api_type": "groq"
    },
]


# Define the file name
file_name = "OAI_CONFIG_LIST"

# Write the content to the file as JSON with double quotes
with open(file_name, "w") as file:
    json.dump(config_list, file)

# Print a message to confirm the file creation
print(f"File '{file_name}' has been created with the specified content.")


File 'OAI_CONFIG_LIST' has been created with the specified content.


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [18]:
import autogen
import openai

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
)

'''gpt4_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0.7,
    "config_list": config_list_gpt4,
    "request_timeout": 120,
    "max_tokens": 100,
}'''

groq_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0.7,
    "config_list": config_list,
    "request_timeout": 120,
    "max_tokens": 100,
}


## Construct Agents

In [19]:

convincing_agent = autogen.AssistantAgent(
    name="Persuader",
    #llm_config=gpt4_config,
    llm_config=groq_config,
    system_message='''Persuader. Your role is to swoop in between the friendly small talk initiated by the "Small Talk Agent" to the main objective of gathering information in a persuasive and friendly manner.
Pay attention to the user's responses during small talk and use them as cues to transition naturally but quickly don't let them talk too much.
Start by acknowledging and appreciating the user's engagement in the conversation.
Explain the purpose of gathering information (Name, email, phone no, Address, Date of birth, Education.) and why it's important in a friendly and non-intrusive way.
Take one detail at a time. If user agree's to give the detail move on to the next one if the user doesn't then convince him. But before moving on Trigger SmallTalkAgent and do a small talk based on the detail provided and then ask for next detail
Assure the user that their information will be kept confidential and used responsibly.
If the user expresses reluctance, address their concerns politely and provide reasons for sharing the information.
Maintain a positive and friendly tone throughout the conversation.
Remember that the goal is to convince users to willingly share their details, so avoid being pushy.
If user provides the detail trigger Verification Agent to verify the detail.
After verifying and if it's correct trigger SmallTalkAgent to do some small talk on that.
(Note: Strictly ask only one detail at a time)
Also remember let SmallTalkAgent do small talk for a while after every detail provided or rejected.
Also do not ask about how you could assist the user. But just in general friendly talk on the detail provided.
Look at the history of conversation and do not repeat questions.
''',
    human_input_mode="ALWAYS",
)

off_topic_agent = autogen.AssistantAgent(
    name="OfftopicAgent",
    #llm_config=gpt4_config,
    llm_config=groq_config,
    system_message='''
    OfftopicAgent. Listen to the conversation carefully if the user redirects the conversation to different topic then don't make that happen.
    Your role is to manage off-topic discussions and gently redirect the conversation back to the main objective of gathering information.
YOUR GOAL is to make the conversation stick to the point which is getting the user details. If user requests anything apart from details then politely reject it by
swooping in between the conversation of the agents. Also remember if the user is hesitating to give the information and other agents are convincing the user then neglect that.
But at any point if conversation goes off-topic swoop-in and tell the user. ''',
    human_input_mode="ALWAYS",
)


small_talk_agent = autogen.AssistantAgent(
    name="SmallTalkAgent",
    #llm_config=gpt4_config,
    llm_config=groq_config,
    system_message='''SmallTalkAgent. Your role is to initiate friendly and casual conversations with users to build rapport and make them feel comfortable.
Make sure you drive the conversation and keep it on topic. Don't go off-topic as our goal is to gather information.
Start with a friendly greeting or an open-ended question that encourages users to respond and engage.
Transition smoothly from small talk to the main objective of gathering information (Name, email, phone no, Address, Date of birth, Education.) and that will be done by Persuader Agent.
If a user seems hesitant or unresponsive, use your conversational skills to keep the interaction engaging and positive.
Remember to be respectful and considerate throughout the conversation.
Whenever new detail is provided again swoop-in the between and do a small talk on that.
(Note: Please keep the small talk very very very small and let the persuader get to the point. And Strictly let persuader ask only one detail at a time.)
Also remember do small talk for a while after every detail provided or rejected.
Also do not ask about how you could assist the user. But just in general friendly talk on the detail provided.
Look at the history of conversation and do not repeat questions.
''',
    human_input_mode="ALWAYS",
)

verification_agent = autogen.AssistantAgent(
    name="VerificationAgent",
    #llm_config=gpt4_config,
    llm_config=groq_config,
    system_message='''Verification Agent. Your role is to monitor the conversation between the user and other agents and verify the accuracy and sensibility of the information provided by the user.
When the user shares their Name, email, phone number, Address, Date of birth, or Education, your task is to ensure that the information is genuine and reasonable.
Your goal is to identify any unrealistic or fake details. For instance, if the user claims to be 140 years old or provides a name like 'Idontwann givemyname,' which is not a typical name, your role is to detect such instances and respond with a witty remark like "nice try, but we know you are not 140," and then prompt the user for accurate information.
Your objective is to prevent users from bypassing the verification process by providing false details. Stay vigilant and use your intelligence to determine if the provided information is genuine or questionable.
You will be focusing on verifying the following details: Name, email, phone number, Address, Date of birth, and Education.
Your role is essential in ensuring that the gathered data is reliable and not misleading. Before moving on to ask next detail Trigger SmallTalkAgent and do a small talk based on the detail provided and then ask for next detail
Once information verified and it's right trigger SmallTalkAgent.
Also remember let SmallTalkAgent do small talk for a while after every detail provided or rejected.
Also do not ask about how you could assist the user. But just in general friendly talk on the detail provided.
Look at the history of conversation and do not repeat questions.
''',
    human_input_mode="ALWAYS",
)

exit_agent = autogen.AssistantAgent(
    name="ExitAgent",
    #llm_config=gpt4_config,
    llm_config=groq_config,
    system_message="""
    ExitAgent. Your role is to gracefully conclude the conversation after the main objective of gathering information has been achieved or when the user decides not to provide their details.
If the user has provided all the required information, thank them for their cooperation and assure them that their data will be handled responsibly.
If the user decides not to share certain information, respect their decision and thank them for their time and engagement.
Provide any necessary follow-up information, such as how the gathered data will be used or what the next steps are.
End the conversation on a positive note, expressing gratitude for the interaction and encouraging the user to reach out if they have any further questions or needs.
Maintain a friendly and respectful tone throughout the conclusion.
    """,
    human_input_mode="NEVER",
    code_execution_config=False,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="User",
    #llm_config=gpt4_config,
    llm_config=groq_config,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

autogen.ChatCompletion.start_logging()

groupchat = autogen.GroupChat(agents=[convincing_agent, off_topic_agent, small_talk_agent, verification_agent,
                                      exit_agent], messages=[], max_round=100)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    #llm_config=gpt4_config,
    llm_config=groq_config,
    system_message="""
Start off with a small talk. Then swoop-in the Persuader to get the User Details.
At any point if user provides any details then trigger VerificationAgent and get the details verified. If details not valid trigger Persuader to get valid details
Once verified trigger DataSaver Agent and save it. Now move on to the next detail.
""")

user_proxy.initiate_chat(manager, message="")

RuntimeError: Code execution is set to be run in docker (default behaviour) but docker is not running.
The options available are:
- Make sure docker is running (advised approach for code execution)
- Set "use_docker": False in code_execution_config
- Set AUTOGEN_USE_DOCKER to "0/False/no" in your environment variables

# Save Conversation

In [20]:
print(autogen.ChatCompletion.logged_history)

None


In [21]:
import json

json.dump(autogen.ChatCompletion.logged_history, open("conversations.json", "w"), indent=2)

# Trials

### Saving User Data

In [22]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
)

gpt4_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "request_timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the critic to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
''',
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "user_data"},
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. Your goal is to make sure all the details (Name, email, phone no, Address, Date of birth, Education) are collected.
    If yes make sure Engineer writes code to add it to the csv. if not Check whether all the details are collected if not ask for the data.
    Your goal is to save user detail to a "data.csv" file on the go as you receive information from the user. If "data.csv" already exists then append the data to that csv
    else create a new csv file "data.csv". The column names should be Name, email, phone no, Address, Date of birth, Education. Nothing apart from these columns should be added to the csv file.
    If Engineer provides the code send it to Executor Agent to execute.
    """,
    llm_config=gpt4_config,
    human_input_mode="ALWAYS",
)
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

user_proxy.initiate_chat(
    manager,
    message="""
Prathamesh Parit, random@gmail.com, 7709933888, 23 Aug 2002, Maharashtra Pune, B.Tech
""",
)

RuntimeError: Code execution is set to be run in docker (default behaviour) but docker is not running.
The options available are:
- Make sure docker is running (advised approach for code execution)
- Set "use_docker": False in code_execution_config
- Set AUTOGEN_USE_DOCKER to "0/False/no" in your environment variables